In [228]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# (X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

# # nopca
# X_train_full = X_train_full.reshape(len(X_train_full), 28*28)
# X_test = X_test.reshape(len(X_test), 28*28)
# X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full)

# pca
pca = PCA(n_components=0.99)
X_train_full_pca = pca.fit_transform(X_train_full.reshape(len(X_train_full), 28*28))
X_test = pca.transform(X_test.reshape(len(X_test), 28*28))
X_train, X_val, y_train, y_val = train_test_split(X_train_full_pca, y_train_full)

print(X_train.shape, y_train.shape)

(45000, 331) (45000,)


In [238]:
def build_model(n_hidden=1, n_nodes=10, lr=0.005, input_shape=(28, 28), output_nodes=10, output_activation='softmax', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'], l1=0.01, l2=0.01, dropout=0.2):
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=input_shape))
  model.add(keras.layers.BatchNormalization())
  for i in range(n_hidden):
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(int(n_nodes / (i + 1)), activation='elu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l1_l2(l1, l2)))
    model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dropout(dropout))
  model.add(keras.layers.Dense(output_nodes, activation=output_activation, kernel_regularizer=tf.keras.regularizers.l1_l2(l1, l2)))
  # nestorov momentum
  optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
  # optimizer = keras.optimizers.RMSprop(learning_rate=lr, rho=0.9)
  # optimizer = keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
  # optimizer = keras.optimizers.Nadam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
  return model

In [239]:
import os

save_dir = os.path.join(os.curdir, 'model.h5')

model = build_model(7, 350, 0.035, X_test.shape[1:], 10, 'softmax', loss=keras.losses.sparse_categorical_crossentropy, metrics=[keras.metrics.sparse_categorical_accuracy], l1=0, l2=0.001, dropout=0.2)
print(model.summary())
model.fit(X_train, y_train, epochs=256, validation_data=(X_val, y_val), callbacks=[
  # keras.callbacks.TensorBoard(get_run_log_dir()),
  keras.callbacks.ModelCheckpoint(save_dir, save_best_only=True),
  keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
  keras.callbacks.ReduceLROnPlateau(factor=0.8,patience=5),
])

Model: "sequential_98"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_98 (Flatten)        (None, 331)               0         
                                                                 
 batch_normalization_734 (Ba  (None, 331)              1324      
 tchNormalization)                                               
                                                                 
 dropout (Dropout)           (None, 331)               0         
                                                                 
 dense_733 (Dense)           (None, 350)               116200    
                                                                 
 batch_normalization_735 (Ba  (None, 350)              1400      
 tchNormalization)                                               
                                                                 
 dropout_1 (Dropout)         (None, 350)             

KeyboardInterrupt: 

In [240]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

loaded = keras.models.load_model(save_dir)
pred = loaded.predict(X_test)
print(confusion_matrix(y_test, np.argmax(pred, axis=1)))
print(accuracy_score(y_test, np.argmax(pred, axis=1)))

# best so far
# pca = PCA(n_components=0.98)
# build_model(6, 300, 0.035, X_test_pca.shape[1:], 10, 'softmax', loss=keras.losses.sparse_categorical_crossentropy, metrics=[keras.metrics.sparse_categorical_accuracy], l1=0.01, l2=0.0005)
# optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
# keras.callbacks.ReduceLROnPlateau(factor=0.8,patience=5)

313/313 [==============================] - 0s 1ms/step
[[ 972    1    1    0    0    0    1    1    3    1]
 [   0 1132    2    0    0    0    0    0    1    0]
 [   7    0 1010    2    0    0    0    6    7    0]
 [   0    0   14  962    0   11    0    8   13    2]
 [   0    0    4    0  961    0    4    0    3   10]
 [   3    1    0    8    0  864    5    2    7    2]
 [  14    3    0    1    4    2  930    0    4    0]
 [   2    6   15    0    1    0    0  991    2   11]
 [   4    1    2    2    1    1    3    4  949    7]
 [   7    2    0    7   11    1    0    7    5  969]]
0.974


In [234]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(confusion_matrix(y_test, pred))
print(accuracy_score(y_test, pred))

[[ 963    0    3    0    1    5    5    1    2    0]
 [   0 1115    6    4    0    0    6    0    4    0]
 [   8    0  952   13   10    1    3    8   34    3]
 [   3    0   11  943    0   16    2   10   19    6]
 [   0    1   10    0  929    2    8    2    7   23]
 [   5    1    6   37   10  806   10    4    8    5]
 [  10    3    5    0    4    4  929    0    3    0]
 [   2   11   20    4    7    0    0  957    2   25]
 [   5    0   13   35   11   24    6    3  865   12]
 [  10    7    5   16   33    2    1   18    7  910]]
0.9369


In [50]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

class_names = [
  'airplane',
  'automobile',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'
]

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
print(X_train_full.shape)

pca = PCA(n_components=0.98)
X_train_full = pca.fit_transform(X_train_full.reshape(len(X_train_full), 32*32*3))
X_test = pca.transform(X_test.reshape(len(X_test), 32*32*3))
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full)
print(X_train_full.shape)
# plt.imshow(X_test[100])
print(class_names[y_test[100][0]])

(50000, 32, 32, 3)
(50000, 444)
deer


In [62]:
import os, time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

log_dir = os.path.join(os.curdir, 'logs')
save_dir = os.path.join(os.curdir, 'cifar.h5')
hidden = 5
neurons = 800
lr = 0.05
l2 = 0
dropout = True
dropout_val = 0.2

def get_run_log_dir():
  run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
  return os.path.join(log_dir, run_id)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=X_test.shape[1:]))
if dropout:
  model.add(keras.layers.Dropout(dropout_val))
model.add(keras.layers.BatchNormalization())
for i in range(hidden):
  if dropout:
    model.add(keras.layers.Dropout(dropout_val))
  model.add(keras.layers.Dense(neurons, kernel_initializer='he_normal', activation='elu', kernel_regularizer=keras.regularizers.l2(l2)))
  model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(len(class_names), activation='softmax', kernel_regularizer=keras.regularizers.l2(l2)))

model.compile(
  optimizer=keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True),
  # optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
  loss=keras.losses.sparse_categorical_crossentropy,
  metrics=(keras.metrics.sparse_categorical_accuracy),
)

print(model.summary())

model.fit(X_train, y_train, epochs=256, validation_data=(X_val, y_val), callbacks=(
  keras.callbacks.TensorBoard(get_run_log_dir()),
  keras.callbacks.ModelCheckpoint(save_dir, save_best_only=True),
  keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True),
  keras.callbacks.ReduceLROnPlateau(factor=0.8, patience=5),
))

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_42 (Flatten)        (None, 444)               0         
                                                                 
 dropout_12 (Dropout)        (None, 444)               0         
                                                                 
 batch_normalization_172 (Ba  (None, 444)              1776      
 tchNormalization)                                               
                                                                 
 dropout_13 (Dropout)        (None, 444)               0         
                                                                 
 dense_529 (Dense)           (None, 800)               356000    
                                                                 
 batch_normalization_173 (Ba  (None, 800)              3200      
 tchNormalization)                                   

KeyboardInterrupt: 

In [60]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

loaded = keras.models.load_model(save_dir)
pred = loaded.predict(X_test)
print(confusion_matrix(y_test, np.argmax(pred, axis=1)))
print(accuracy_score(y_test, np.argmax(pred, axis=1)))

313/313 [==============================] - 1s 3ms/step
[[610   7  72  19  33  15  23   9 169  43]
 [ 82 491  25  27  17  23  37  20 143 135]
 [ 82   1 460  36 178  90  92  28  26   7]
 [ 30   5 120 202 128 284 155  30  27  19]
 [ 50   1 131  29 579  50  82  49  23   6]
 [ 21   3 117 105 124 519  55  42   9   5]
 [  6   5  65  19 144  48 692   7   8   6]
 [ 23   4  51  45 146 104  37 552  16  22]
 [ 90  15  24  11   9  19  22   5 773  32]
 [ 48  89  25  36  28  37  39  42 132 524]]
0.5402


In [49]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(confusion_matrix(y_test, pred))
print(accuracy_score(y_test, pred))

/var/folders/sw/q7k30xcj51x3tc06z506z3n00000gn/T/ipykernel_70935/1308907044.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


[[555  60  49  28  19  26  30  24 148  61]
 [ 40 579  23  32  12  25  23  28  67 171]
 [107  36 310  79 150  69 129  59  38  23]
 [ 55  49  98 271  51 180 123  63  29  81]
 [ 57  27 136  64 375  59 150  81  28  23]
 [ 35  43  86 164  60 373  94  75  38  32]
 [ 12  33  92  59 109  60 555  26  18  36]
 [ 41  47  49  62  75  98  70 432  34  92]
 [114  92  17  22  17  41  18  17 594  68]
 [ 48 190  17  43   9  34  34  37  84 504]]
0.4548
